<a href="https://colab.research.google.com/github/Teja5164/Chess_pieces_detectionusingYOLOv8/blob/main/chess_pieces_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-chess torch torchvision opencv-python-headless matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 48.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=1332783f4ebef53465cfb9f3b5454ead61a72601372dcf36db03281a535719d8
  Stored in directory: /root/.cache/pip/wheels/83/1f/4e/8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess


In [ ]:
!pip install ultralytics opencv-python-headless matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.2 MB/s eta 0:00:00


In [ ]:
from google.colab import files

print("Upload your chess piece classification model (.pt):")
uploaded_model = files.upload()
model_path = list(uploaded_model.keys())[0]
print(f"Uploaded model: {model_path}")

print("\nNow upload your chessboard images (.jpg):")
uploaded_images = files.upload()
image_files = list(uploaded_images.keys())
print(f"Uploaded images: {image_files}")


Upload your chess piece classification model (.pt):


Saving yolo8-20epoch.pt to yolo8-20epoch.pt
Uploaded model: yolo8-20epoch.pt

Now upload your chessboard images (.jpg):


Saving 1.jpeg to 1.jpeg
Saving 2.jpeg to 2.jpeg
Saving 3.jpeg to 3.jpeg
Saving 4.jpeg to 4.jpeg
Saving 5.jpeg to 5.jpeg
Uploaded images: ['1.jpeg', '2.jpeg', '3.jpeg', '4.jpeg', '5.jpeg']


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

model = YOLO(model_path)
print("YOLOv8 model loaded.")


YOLOv8 model loaded.


In [ ]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def detect_and_warp_board(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        raise Exception("No board contour found")
    max_contour = max(contours, key=cv2.contourArea)
    peri = cv2.arcLength(max_contour, True)
    approx = None
    for eps in np.linspace(0.01, 0.05, 5):
        tmp = cv2.approxPolyDP(max_contour, eps * peri, True)
        if len(tmp) == 4:
            approx = tmp
            break
    if approx is None or len(approx) !=4:
        x,y,w,h = cv2.boundingRect(max_contour)
        approx = np.array([[[x,y]],[[x+w,y]],[[x+w,y+h]],[[x,y+h]]], dtype=np.int32)
    pts = approx.reshape(4,2).astype(np.float32)
    rect = order_points(pts)
    size = 400
    dst = np.array([[0,0],[size-1,0],[size-1,size-1],[0,size-1]], dtype=np.float32)

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(img, M, (size, size))
    return warped


In [ ]:
def split_board_into_squares(board_img):
    squares = []
    square_size = board_img.shape[0] // 8
    for r in range(8):
        row_squares = []
        for c in range(8):
            sq = board_img[r*square_size:(r+1)*square_size, c*square_size:(c+1)*square_size]
            row_squares.append(sq)
        squares.append(row_squares)
    return squares


In [ ]:
piece_to_fen = {
    'white_king': 'K', 'white_queen': 'Q', 'white_rook': 'R', 'white_bishop': 'B', 'white_knight': 'N', 'white_pawn': 'P',
    'black_king': 'k', 'black_queen': 'q', 'black_rook': 'r', 'black_bishop': 'b', 'black_knight': 'n', 'black_pawn': 'p'
}

def generate_fen(board):
    fen_rows = []
    for row in board:
        fen = ''
        empty = 0
        for sq in row:
            if sq == '.':
                empty += 1
            else:
                if empty > 0:
                    fen += str(empty)
                    empty = 0
                fen += sq
        if empty > 0:
            fen += str(empty)
        fen_rows.append(fen)
    return '/'.join(fen_rows) + ' w - - 0 1'


In [ ]:
for img_file in image_files:
    print(f"Processing {img_file} ...")
    img = cv2.imread(img_file)
    try:
        warped_board = detect_and_warp_board(img)
        squares = split_board_into_squares(warped_board)
        size = warped_board.shape[0]
        sq_size = size // 8

        results = model(warped_board)[0]  # Detect on warped image

        # Initialize empty board
        board = [['.' for _ in range(8)] for __ in range(8)]

        for box in results.boxes.data.cpu().numpy():
            x1, y1, x2, y2, conf, cls = box
            cls = int(cls)
            class_name = results.names[cls]

            cx = (x1 + x2) / 2
            cy = (y1 + y2) / 2

            col = int(cx // sq_size)
            row = int(cy // sq_size)

            fen_char = piece_to_fen.get(class_name, '.')
            if 0 <= row < 8 and 0 <= col < 8:
                board[row][col] = fen_char

        fen = generate_fen(board)
        print(f"FEN for {img_file}: {fen}")

        fen_filename = img_file.rsplit('.', 1)[0] + '.fen'
        with open(fen_filename, 'w') as f:
            f.write(fen)
        print(f"Saved FEN to {fen_filename}\n")

    except Exception as e:
        print(f"Error processing {img_file}: {e}\n")


Processing 1.jpeg ...

0: 640x640 1 black-bishop, 2 black-kings, 4 black-knights, 8 black-pawns, 1 black-queen, 1 white-bishop, 1 white-king, 3 white-knights, 9 white-pawns, 2 white-queens, 3 white-rooks, 233.0ms
Speed: 9.5ms preprocess, 233.0ms inference, 5.9ms postprocess per image at shape (1, 3, 640, 640)
FEN for 1.jpeg: 8/8/8/8/8/8/8/8 w - - 0 1
Saved FEN to 1.fen

Processing 2.jpeg ...

0: 640x640 2 black-kings, 3 black-knights, 7 black-pawns, 1 black-rook, 1 white-bishop, 2 white-knights, 7 white-pawns, 1 white-queen, 3 white-rooks, 212.0ms
Speed: 5.2ms preprocess, 212.0ms inference, 4.8ms postprocess per image at shape (1, 3, 640, 640)
FEN for 2.jpeg: 8/8/8/8/8/8/8/8 w - - 0 1
Saved FEN to 2.fen

Processing 3.jpeg ...

0: 640x640 1 black-bishop, 1 black-king, 3 black-knights, 6 black-pawns, 1 black-rook, 2 white-bishops, 2 white-kings, 2 white-knights, 5 white-pawns, 1 white-rook, 207.4ms
Speed: 4.5ms preprocess, 207.4ms inference, 4.1ms postprocess per image at shape (1, 3, 64